 <img src="Images/fc.pos.jpg" class="bg-primary" align = 'left' width=200/>

# Academia de Física 2026
### Exemplo: Caudais do Rio Mondego

Para obter dados podemos começar com a seguinte pergunta ao Chatgpt:
 - *onde posso encontrar dados de caudais de rios em Portugal?*
 
A primeira resposta envia-nos para o SNIRH – Sistema Nacional de Informação de Recursos Hídricos.
O site é [SNIRH-APA](https://snirh.apambiente.pt/).
Etse site não é muito claro, mas clicando em `Dados de Base->Monitorização` e selecionando
 - Redes: hidromética
 - Bacias Hidrográficas: Mondego 
 - `Aplicar filtros`
 
Podemos agora selecionar a estação de `Açude Ponte de Coimbra` e a série de dados `Caudal descarregado médio diário` e ao clicar `Ver/Guardar dados` surge a série  de dados com registo diário de caudais desde 1987 a 2024, que podemos exportar em formato csv. Abre o ficheiro e guarda-o em formato `EXCEL` (.xlsx) e estás pronto a começar a análise.
 

In [ ]:
#Começa sempre por executar esta célula
#%matplotlib inline                                      
import numpy as np                    # modulo numérico , essencial para trabalho cientifico; chamado
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st


In [ ]:
mondego=pd.read_excel("Dados/serie_Mondego.xlsx")
mondego

É mais fácil começar por eliminar no EXCEL as três primeiras linhas e as últimas que são texto de identificação da origem dos dados: Convém tambem escrever o cabeçalho da primeira coluna **DATA**. Depois dessa alteração voltamos a importar os dados:

In [ ]:
mondego=pd.read_excel("Dados/serie_Mondego2.xlsx")
mondego


O que nos interessa é a coluna **descarregado**

In [ ]:
mondego=mondego["descarregado"]
mondego

Podemos começar com uma "vista de olhos" aos dados: o método `describe()` dá informações básicas sobre o número de dados (dias) -- `count`--,  a média dos valores -- `mean`--, etc.  

In [ ]:
#estatísticas básicas
mondego.describe()

Temos 13314 dias de observação. O máximo caudal foi de 1902 m³/s, mas a média é apenas 68 m³/s  e o percentil 50% apenas 19.5 m³/s. Isto é metade das observações tem valores iguais ou inferiores a 19.5 m³/s. As cheias, o que nos interessa mais,  são raras mas com caudais várias vezes superiores à média.Para estudar cheias vamos concentrar-nos nos valores máximos em cada intervalo de 30 dias. Em dias sucessivos os caudais são muito semelhantes. Considerando intervalos de 30 dias e selecionando oa máximos de cada intervalo devemos ter valores mais ou menos independentes. 

Quantos meses temos?

In [ ]:
nmeses=int(mondego.count()/30)
print(nmeses)

Vamos criar um `array` com 443 entradas para guardar os caudais máximos de cada 30 dias:

In [ ]:
caudais_mensais=np.zeros(443)
c1=mondego.loc[0:29].max()          # primeiro mês
c2=mondego.loc[30:30+29].max()      # segundo mês
print(c1,c2)

Com um simples ciclo `for` conseguimos preencher o `array` `caudais_mensais` com os valores máximos de cada 30 dias.

In [ ]:
mes=30
for i in range(443):
    caudais_mensais[i]=mondego.loc[i*mes:i*mes+29].max()

Agora criamos uma série de dados (Pandas) com o array de máximos de caudais mensais

In [ ]:
mondego=pd.Series(caudais_mensais)
mondego

<div class="alert alert-block alert-warning">
    <b>Exercício 1</b>
    <p> Calcula o número de anos de observação destes registos. Repete a operação acima para criar uma série de dados com os caudais máximos anuais</p>
</div>

In [ ]:
# faz aqui o teu exercicio


In [ ]:
mondego.describe()

Temos 443  observações. O máximo caudal foi de 1902 m³/s, mas a média é apenas 160 m³/s  e o percentil 50% apenas 58.3 m³/s. Isto é, metade das observações tem valores iguais ou inferiores a 58,3 m³/s. As cheias, o que nos interessa mais,  são raras mas com caudais várias vezes superiores à média.P

Um gráfico usando diretamente os métodos gráficos do `Pandas`

In [ ]:
mondego.plot(ms=0,lw=1) #ms é marker size e lw linewidth
plt.grid(True)    

Estamos interessados nos eventos de cheias, que ocorrem raramente, mas em que os caudais são muito superiores ao "normal". Podemos perguntar, por exemplo, qual é o caudal que é ultrapassado apenas uma vez em 100 - probabilidade de $p=0.01$. Os `dataframe`  tem um método para obter esses valores. Mas antes disso vamos investigar esses valores, contando em quantos meses o  caudal é superior a um limite. Isso ilustra a funcionamento de uma *máscara* para extrair valores de um `dataframe`que cumprem um dado critério. Vê o exemplo seguinte:

In [ ]:
x=np.linspace(0,9,10)
mask=x>4
mask

In [ ]:
x[mask]

Como vês, quando o valor de um elemento de  `mask` é `False` o valor respetivo  é retirado do array `x`. 

<div class="alert alert-block alert-warning">
    <b>Exercício 2</b>
    <p> Com base no expemplo acima, obtém a lista de valores do <i>dataframe</i> <b>mondego</b> que ultrapassam 1000 m³ /s e calcula a frequência relativa (estimador da probabilidade).</p>
</div>

In [ ]:
# faz aqui o teu exercício

<div class="alert alert-block alert-warning">
    <b>Exercício 3</b>
    <p> Com um ciclo,  repete o cálculo acima para uma lista de caudais [1000, 1900] por intervalos de 100.</p>
</div>

In [ ]:
# faz aqui o teu exercício

Se fizeste o exercício deves notar que o valor de 1000 m³ /s  deve estar próximo do percentil 98%.

No `Pandas` o percentil $q$ é obtido com o método `quantile(q)`. 

In [ ]:
mondego.quantile(0.98)

<div class="alert alert-block alert-warning">
    <b>Exercício 4</b>
    <p> Usando o método da máscara determina a frequência de ocorrência de cheias de caudal superior a 998 m³/s. Verifica se a frequência relativa é próxima 0.02</p>
</div>

In [ ]:
# faz aqui o teu exercício

Qual é a probabilidade de acontecer uma cheia destas num ano? 

Um ano tem 12 meses. A cheia pode não ocorrer, ocorrer uma vez, duas ,....  A pergunta mais fácil de responder é a seguinte: 
 - Qual é a probabilidade de não acontecer uma destas cheias num ano? Ou seja, não ocorrer em 12 meses consecutivos?
 
Pensa num dado. Qual a probabilidade de não sair 6 num lançamento? Obviamente 5/6. E de não sair um único 6 em dois lançamentos? Há 5 possibilidades de não sair no primeiro e 5 no segundo. Por isso em 36 possibilidades para os dois lançamentos, há 25 em que não sai um único 6, ou seja:

- probabilidade de não haver 6 em dois lançamentos:

$$\frac{25}{36}=\frac{5}{6}\times \frac{5}{6}$$ 

A probabilidade de não acontecer uma cheia de caudal superior a 998 m³/s , num mês é $q=0.98$; probabilidade de não acontecer  em nenhum de dois meses consecutivos é $q\times q=q^2$. Em $n$ meses $q^n$. Logo, a probabilidade de ocorrer pelo menos uma cheia  em $n$ meses é $1-q^n$, porque a probabilidade de acontecer $0$ ou $1\dots$ ou 12 é 1. (estamos a supor eventos independentes, o que não é irrealista para caudais mensais).

<div class="alert alert-block alert-warning">
    <b>Exercício 5</b>
    <p> Calcula a probabilidade de pelo menos uma cheia de caudal superior a 998 m³ /s ocorrer num ano. Avalia o resultado contando as cheias deste tipo ocorridas no tempo de observação</p>
</div>

In [ ]:
# faz aqui o teu exercício

Será que a frequencia relativa de anos com  cheias superiores a  998 m³ /s é próxima deste valor?

In [ ]:
mask=mondego>mondego.quantile(0.98)
mondego[mask].count()/(mondego.count()/12)

Se uma cheia de caudal superior a $C$ ocorre com probabilidade $p$ num ano, em média quando tempo decorre entre tais cheias? O número médio de cheias em $N$ anos é 
$$
\langle n\rangle=pN 
$$
Temos em média uma cheia num número $N$ de anos em que  $pN$=1. Ou seja $N_r$, *tempo de recorrência*,  vale $N_r=1/p$.

<div class="alert alert-block alert-warning">
    <b>Exercício 6</b>
    <p> Define uma função de argumento real, $q$ que te dê o tempo de recorrência de cheias de caudal superior ao percentil $1-q$. Calcula a função para $q= 2\times 10^{2}$,  $q=10^{-2}$ e $5\times 10^{-3}$. </p>
</div>

In [ ]:
#faz aqui o teu exercicio

Por esta estimativa cheias de valor superior a 1793 m³/s devem ocorrer de 42 em em 42 anos. No periodo em análise (36 anos)  quantas vezes se verificaram?

In [ ]:
q=2e-3
mask=mondego>mondego.quantile(1-q)
mondego[mask].count()

### Explorar a distribuição
Podemos tentar  descobrir a distribuição de caudais para valores extremos. Mas como viste, o número de observações de cheias é reduzido e por isso o que se segue é altamente espculativo. Mas em séries de dados mais longas poderia dar algumas indicações úteis. 

Começamos por representar um histograma cumulativo.
Para fazer um hsitograma definimos um conjunto de intervalos de caudais (40 neste exemplo) e determinamos a frequência relativa de ocorrência de caudais em cada intervalo. Num histograma cumulativo,  a altura de cada barra é a percentagem de vezes que se observa um valor de caudal inferior ao valor da abicssa. 

In [ ]:
# o histograma devolve 3 listas
#
#   hheights: frequência relativa em cada bin
#   bins; limite inferior de cada intervalo
#   patches_ elementos gráficos
#   Se o histograma for cumulativo a altura de cada bin é a frequência relativa
#   somada dos intervalos até ao intervalo  em causa.

hheights,bins,patches =plt.hist(mondego, bins=40, rwidth=.80, cumulative=True, density=True)
plt.grid(True)
#plt.ylim([.99,1])

In [ ]:
# limite inferiores de cada bin
bins

In [ ]:
#ordenada correspondente a cada bin
np.set_printoptions(precision=6)
hheights

In [ ]:
# um grafico em que as abcissas são o ponto médio de cada intervalo (bin)
cd=(bins[1:]+bins[:-1])/2      # o ponto médio de cada bin
plt.plot(cd,hheights,'bs',ms=2)
plt.grid('True')
plt.xlabel(r'$m^3\  s^{-1}$')
plt.ylabel('freq acum')
#plt.ylim([.95,1])

Vamos fazer um ajuste a uma função com dois parâmtros. Este função foi escolhida por ter uma comportamento parecido com o do gráfico acima. Não te preocupes com a expressão. Basta que reconheças que tem dois parâmetros livres para ajustar, `beta`  e `mu`. 

In [ ]:
from scipy.optimize import curve_fit     # ajuste não linear
def model(x,beta,mu):
    maximo=mondego.max()
    return np.tanh(beta*(x-mu)/maximo)

Temos 40 dados. Como estamos interessados podemos selecionar apenas parte dos dados correpondente ao valores mais altos. Para isso definimo um variável `cutoff` que elimina os dados de valor mais baixo. 

In [ ]:
x=np.linspace(0,39,40)
x[10:]

In [ ]:
# ajuste Parms tem os valores dos dois parametros
cutoff=10
parms,cov=curve_fit(model,cd[cutoff:],hheights[cutoff:])
parms,cov

In [ ]:
# gráfico conjunto
plt.plot(cd[cutoff:],hheights[cutoff:],'bs', cd[cutoff:], model(cd[cutoff:],parms[0],parms[1]),'r-',ms=2)
plt.grid('True')
plt.xlabel(r'$m^3\  s^{-1}$')
plt.ylabel('freq acum')

Usando o modelo podemos estimar a probabilidade de ter um caudal de 2000 m³ /s,  o valor crítico que tanto assustou a proteção civil. Na série de dados obtida esse valor nunca foi observado.

In [ ]:
pext = model(2000,parms[0],parms[1])
pext

In [ ]:
mondego.quantile(pext)

In [ ]:
recorrencia(1-pext)

In [ ]:
msk=mondego>mondego.quantile(pext)
mondego[msk].count()

O tempo de recorrência para este valor extremo (com cutoff =10)  é de 96 anos e no período de observação, foi observado uma vez. AVISO: estamos a lidar com eventos muito raros e as estatísticas são pouco fiáveis. 

<div class="alert alert-block alert-warning">
    <b>Exercício 7</b>
    <p> Investiga o que varia com deixas variar o cutoff </p>
</div>

In [ ]:
# faz aqui o exercicio.